How to read data and store it in an Xarray?

- Try PyroSAR
- Try xarray-ceos reader (if pyrosar does not work)
- Look for alternatives 
    - sarpy, reads SLC and sicd
    - envi reader (for which data?)

- Found out: gdal has a `JAXAPALSAR` driver
- We can access the data directly with `rioxarray`

In [ ]:
# TODO: add pyroSAR as dependency if needed
# import pyroSAR

In [ ]:
import rioxarray as riox
import xarray as xr
import dask.array as da
data_path = "/data/psp/SAN_FRANCISCO_ALOS1/"

In [ ]:

# open with chunks to use dask backend
S_elts = riox.open_rasterio("/data/psp/SAN_FRANCISCO_ALOS1/VOL-ALPSRP202350750-P1.1__A", chunks=(4, 1024, 1024)).transpose("x", "y", "band")
S_elts.coords["band"] = ["HH", "HV", "VH", "VV"]

In [ ]:
S_elts[...,3].mean().compute()

In [ ]:
# convert S to C3

# coordinates of the Hermitian covariance matrix
# we only store the upper part
C3_coords = (S_elts.coords["y"], S_elts.coords["x"], ["11", "12", "13", "22", "23", "33"])

# scattering vector 
k_hh = S_elts[:,:,0]
k_vv = S_elts[:,:,3]
k_hv = 0.5*(S_elts[:,:,1] + S_elts[:,:,2])

# compute the cross products
C3_elts = []
for i, e1 in enumerate([k_hh, k_hv, k_vv]):
    for j, e2 in enumerate([k_hh, k_hv, k_vv]):
        if j >= i:
            C3_elts.append(e1 * e1.conj())

# stack elements
C3_data = da.stack(C3_elts).T

# make a lazy DataArray
C3 = xr.DataArray(data=C3_data.rechunk((1024, 1024, 6)), dims = ("y", "x", "elt"), coords=C3_coords, name="C3")

In [ ]:
da.mean(C3[...,1]).compute()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
plt.imshow(abs(C3[..., 0]), interpolation="none", vmax=100)
plt.colorbar(fraction=0.046, pad=0.04)